In [1]:
import pandas as pd 

In [2]:
def extract_experiments(experiment_states):
    for exp_state in experiment_states:
        if exp_state.rev == 'workspace':
            continue
            
        experiments = exp_state.experiments or []

        for experiment in experiments:
            revs = experiment.revs or []
            for rev in revs:
                params = next(iter(rev.data.params.values()), {}).get("data")
                metrics = next(iter(rev.data.metrics.values()), {}).get("data")
                yield {
                    'commit': exp_state.rev,
                    'id': rev.rev,
                    'name': rev.name,
                    'params': params,
                    'metrics': metrics,
                }

In [3]:
from dvc.repo import Repo

# Initialize a DVC Repo object
repo = Repo()

# Get the experiments
experiments = list(extract_experiments(repo.experiments.show(num=100)))

In [ ]:
df = (
    pd.json_normalize(experiments)
    .sort_values(["params.model", "params.prompt", "params.temperature", "params.run"])
    .drop(columns=["commit", "id"])
)
# mask = df["params.model"].str.contains("tgi")
# df.loc[mask]
df